In [11]:
import math
import copy
fread = open("Q3Q3.txt", "r") #netdata.txt
G_v_e = fread.readline()
G_v_e = G_v_e.split(' ') #change to \t if tab spaced
V = int(G_v_e[0])
E = int(G_v_e[1].strip())
V_list = list(range(0,V,1))
lines = fread.readlines()
A_dict = {}
G_mat = [ [ 0 for i in range(4) ] for j in range(E) ]
g = 0
for line in lines:
    t_h = line.split(' ') #change to \t if tab spaced
    print(t_h)
    G_mat[g][0] = int(t_h[0])
    G_mat[g][1] = int(t_h[1])
    G_mat[g][2] = int(t_h[2])
    G_mat[g][3] = int(t_h[3].strip())
    g += 1
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    if tail in A_dict.keys():
        A_dict[tail].append(head)        
    else:
        A_dict[tail] = [head]

capacity = [[0 for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    capacity[tail][head] = G_mat[i][3]

print(G_mat, "\n")
print(A_dict)   
print(capacity)
fread.close()

['0', '1', '1', '27\n']
['0', '4', '1', '21\n']
['1', '2', '1', '27\n']
['1', '4', '1', '12\n']
['1', '5', '1', '16\n']
['2', '3', '1', '14\n']
['2', '6', '1', '16\n']
['3', '7', '1', '19\n']
['4', '5', '1', '14\n']
['4', '8', '1', '26\n']
['4', '9', '1', '12\n']
['5', '6', '1', '27\n']
['5', '8', '1', '8\n']
['5', '9', '1', '16\n']
['5', '10', '1', '10\n']
['6', '7', '1', '23\n']
['6', '9', '1', '15\n']
['7', '10', '1', '7\n']
['7', '11', '1', '28\n']
['8', '9', '1', '28\n']
['8', '12', '1', '29\n']
['8', '13', '1', '19\n']
['9', '12', '1', '9\n']
['9', '13', '1', '17\n']
['9', '14', '1', '12\n']
['10', '13', '1', '15\n']
['10', '14', '1', '20\n']
['11', '15', '1', '19\n']
['12', '13', '1', '23\n']
['12', '16', '1', '25\n']
['12', '17', '1', '15\n']
['13', '14', '1', '15\n']
['13', '17', '1', '25\n']
['14', '15', '1', '14\n']
['14', '18', '1', '15\n']
['15', '18', '1', '7\n']
['15', '19', '1', '18\n']
['16', '17', '1', '13\n']
['16', '20', '1', '15\n']
['17', '18', '1', '22\n']
['17',

In [12]:
def create_residual_network(G_mat, A_dict, capacity):
    R = copy.deepcopy(G_mat)
    for i in A_dict.keys():
        for j in A_dict[i]:
            if (j in A_dict.keys()):
                if (i in A_dict[j]):
                    continue
            R.append([j, i, 0, 0])
    return R

def rev_Dijkstra(A_dict, cost, V, s, t):
    #print('rev D')
    parent = {}
    for i in range(V):
        parent[i] = i
    dist = [math.inf]*V
    dist[s] = 0
    fixed = [s]
    chosen = s
    while (t not in fixed):
        if (chosen not in A_dict.keys()):
            #print("cant use DJ here")
            return dist[t], parent
        chosen_range = A_dict[chosen]
        #print(chosen_range)
        k = -1
        #print('previous d', dist)
        for i in range(len(chosen_range)):
            j = chosen_range[i]
            if (j in fixed):
                continue
            if (dist[j] >= dist[chosen] + cost[chosen][j]):
                dist[j] = dist[chosen] + cost[chosen][j]
                parent[j] = chosen
        #print('modified d', dist)
        min_d = 1e8
        min_v = -1
        for v in range(V):
            if ((dist[v] <= min_d) and (v not in fixed) and (dist[v] != math.inf) and (v in A_dict.keys() or v == t)):
                min_d = dist[v]
                min_v = v
        k = min_v
        if (k == -1 or k == math.inf):
            break
        else:
            fixed.append(k)
            chosen = k
        #print(chosen)
    #print(dist)
    return dist, parent


def set_distance_labels(R, capacity, s, end):
    mod_cost = [[math.inf for i in range(V)] for j in range(V)]
    R_dict = {}
    # Develop modified cost & dictionary
    for e in range(len(R)):
        t = R[e][0]
        h = R[e][1]
        if (capacity[t][h] != 0):
            mod_cost[h][t] = 1
            if (h in R_dict.keys()):
                if (t not in R_dict[h]):
                    R_dict[h].append(t)
            else:
                R_dict[h] = [t]
        else:
            if (h in R_dict.keys()):
                if (t in R_dict[h]):
                    R_dict[h].remove(t)
    
    dist, parent = rev_Dijkstra(R_dict, mod_cost, V, end, s)
        
    return dist

    

In [22]:
def retrieve_flow(G_mat, R, capacity):
    flow = []
    for i in range(E):
        t = G_mat[i][0]
        h = G_mat[i][1]
        old = G_mat[i][3]
        new = capacity[t][h]
        if (old > new):
            flow.append(old - new)
        else:
            flow.append(0)            
    return flow

def push(R, capacity, active, excess, dist, v, j, s, t):
    '''
    for v in active:
        cond = 0
        for j in A_dict[v]:
            if ((dist[v] - dist[j]) == 1):
                cond = 1
                break
        if (cond):
            break
    if (cond == 0):
        return
    '''
    #print(v, j)
    f = min(excess[v], capacity[v][j])
    #print(f)
    excess[v] = excess[v] - f
    if ((j != s) and (j != t)):
        excess[j] = excess[j] + f
        if (j not in active):
            active.append(j)
    capacity[v][j] = capacity[v][j] - f
    capacity[j][v] = capacity[j][v] + f
    if (excess[v] == 0):
        active.remove(v)
    #print('after push status')
    #print(excess)
    #print(active)
    #for i in range(len(capacity[0])):
    #    print(capacity[i])
    return

# Develop modified dictionary
def mod_dict(R, capacity):
    R_dict = {}
    for e in range(len(R)):
        t = R[e][0]
        h = R[e][1]
        if (capacity[t][h] != 0):
            if (t in R_dict.keys()):
                if (h not in R_dict[t]):
                    R_dict[t].append(h)
            else:
                R_dict[t] = [h]
        else:
            if (t in R_dict.keys()):
                if (h in R_dict[t]):
                    R_dict[t].remove(h)
    return R_dict
   
    
def relabel(R, capacity, active, dist):
    R_dict = mod_dict(R, capacity)
    min_dist = math.inf
    v_min = -1
    for v in active:
        if (dist[v] < min_dist):
            min_dist = dist[v]
            v_min = v
    wmin = math.inf
    #print(v_min)
    flag = 0
    for j in R_dict[v_min]:
        if ((dist[j] <= wmin) and (capacity[v_min][j] > 0)):
            wmin = dist[j]
            flag = 1
    if (flag):
        dist[v_min] = wmin + 1
    return
        

def preflow_push(G_mat, A_dict, capacity, s, t):
    R = create_residual_network(G_mat, A_dict, capacity)
    dist = set_distance_labels(R, capacity, s, t)
    flow = []
    if (dist[s] == math.inf):
        print(t, 'is not connected to', s)
        return flow
    dist[s] = V
    excess = [0 for i in range(V)]
    active = []
    for j in A_dict[s]:
        active.append(j)
        excess[j] = capacity[s][j]
        capacity[j][s] = capacity[s][j]
        capacity[s][j] = 0
    itr = 0
    
    #print values
    '''
    print(active)
    print(excess)
    print(dist)
    print('capacity')
    for i in range(V):
        print(capacity[i])
    '''
        
    #print('start of while')
    while(len(active) != 0):
        #print('here')
        #print(active)
        itr = itr + 1
        #if (itr > 15):
        #   break
        cond = 0       
        R_dict = mod_dict(R, capacity)
        for j in active:
            cond = 0
            if (j not in R_dict.keys()):
                break
            for i in R_dict[j]:
                if ((dist[j] - dist[i]) == 1 and capacity[j][i] > 0):
                    cond = 1
                    break
            if (cond):
                break
        if (cond):
            push(R, capacity, active, excess, dist, j, i, s, t)
        else:
            #print('no push')
            #print(dist)
            relabel(R, capacity, active, dist)
            #print(dist)
    flow = retrieve_flow(G_mat, R, capacity)
    return flow
            
        
    

In [23]:
s = 1
t = 22
capacity = [[0 for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    capacity[tail][head] = G_mat[i][3]
    
R = create_residual_network(G_mat, A_dict, capacity)
dist = set_distance_labels(R, capacity, s, t)
#last resort - s
dist[s] = V
print(dist)

[5, 24, 5, 5, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 3, 2, 1, inf, inf, inf, 0, inf]


In [24]:
s = 1
t = 22
capacity = [[0 for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    capacity[tail][head] = G_mat[i][3]


flow = preflow_push(G_mat, A_dict, capacity, s, t)
#printing flow through arcs
print('Flow through arcs-')
flow_value = 0
for i in range(E):
    print('(', G_mat[i][0], ',' , G_mat[i][1], ')', ':', flow[i])
    if (G_mat[i][0] == s):
        flow_value = flow_value + flow[i]
    
# Flow_value
print('Max flow: ', flow_value)

Flow through arcs-
( 0 , 1 ) : 0
( 0 , 4 ) : 0
( 1 , 2 ) : 10
( 1 , 4 ) : 12
( 1 , 5 ) : 5
( 2 , 3 ) : 0
( 2 , 6 ) : 10
( 3 , 7 ) : 0
( 4 , 5 ) : 12
( 4 , 8 ) : 0
( 4 , 9 ) : 0
( 5 , 6 ) : 9
( 5 , 8 ) : 8
( 5 , 9 ) : 0
( 5 , 10 ) : 0
( 6 , 7 ) : 7
( 6 , 9 ) : 12
( 7 , 10 ) : 7
( 7 , 11 ) : 0
( 8 , 9 ) : 5
( 8 , 12 ) : 3
( 8 , 13 ) : 0
( 9 , 12 ) : 9
( 9 , 13 ) : 8
( 9 , 14 ) : 0
( 10 , 13 ) : 7
( 10 , 14 ) : 0
( 11 , 15 ) : 0
( 12 , 13 ) : 0
( 12 , 16 ) : 12
( 12 , 17 ) : 0
( 13 , 14 ) : 6
( 13 , 17 ) : 9
( 14 , 15 ) : 6
( 14 , 18 ) : 0
( 15 , 18 ) : 6
( 15 , 19 ) : 0
( 16 , 17 ) : 12
( 16 , 20 ) : 0
( 17 , 18 ) : 21
( 17 , 20 ) : 0
( 17 , 21 ) : 0
( 18 , 19 ) : 0
( 18 , 21 ) : 0
( 18 , 22 ) : 27
( 18 , 23 ) : 0
( 19 , 23 ) : 0
( 20 , 21 ) : 0
( 22 , 23 ) : 0
Max flow:  27
